# Join features

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
ss = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

# Add S3 paths 

In [ ]:
mfcc = ss.read.csv('mfcc_final.csv', header=True)
cqt = ss.read.csv('cqt_final.csv', header=True)
gender = ss.read.csv('Lab41-SRI-VOiCES-speaker-gender-dataset.csv', header=True)

In [8]:
gender.show()

+-------+------+---------------+
|Speaker|Gender|        DataSet|
+-------+------+---------------+
|     32|     F|train-clean-100|
|     83|     F|train-clean-100|
|     93|     F|train-clean-360|
|    112|     F|train-clean-360|
|    115|     F|train-clean-360|
|    118|     M|train-clean-100|
|    122|     M|train-clean-360|
|    150|     F|train-clean-100|
|    154|     M|train-clean-360|
|    159|     M|train-clean-360|
|    166|     F|train-clean-360|
|    174|     M|      dev-clean|
|    175|     F|train-clean-360|
|    176|     M|train-clean-360|
|    188|     F|train-clean-360|
|    192|     F|train-clean-360|
|    196|     M|train-clean-100|
|    198|     F|train-clean-100|
|    200|     F|train-clean-100|
|    204|     M|train-clean-360|
+-------+------+---------------+
only showing top 20 rows



In [4]:
# define a udf that extracts the speaker id out of the filename
def get_speaker(filename):
    i = filename.index('sp')
    return int(filename[i+2:i+6])
speaker = udf(get_speaker)

In [5]:
# process speaker id
cqt = cqt.withColumn('Speaker', speaker('FileName')).drop('FileName', '_c0')
mfcc = mfcc.withColumn('Speaker', speaker('FileName')).drop('FileName', '_c0')

In [6]:
# join the dataframe
results = gender.join(mfcc, on='Speaker', how='rightOuter')\
                .join(cqt, on='Speaker', how='rightOuter')

In [7]:
results.show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+--------------------+-------------------+--------------------+------+---------------+
|Speaker|          Centroid|          variance|          skewness|          kurtosis|              mfcc1|             mfcc2|              mfcc3|             mfcc4|              mfcc5|              mfcc6|              mfcc7|              mfcc8|              mfcc9|             mfcc10|             mfcc11|             mfcc12|      roll_off_max|      roll_off_min|            flatness|   zeroCrossingRate|                 rms|Gender|        DataSet|
+-------+------------------+------------------+------------------+------------------+-------------------+-

In [44]:
# check the number of observations
print(features.count())
print(results.count())

9381
9381


In [45]:
sc.stop()